In [1]:
from tkinter import filedialog,Tk,Button,Label
from tkinter import *
from PIL import Image


from PIL import ImageTk

#import cv2

from keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools
print("hello")


hist = 0
model = load_model('trained_model_fake_real.h5')
class_names = ['fake', 'real']
image_size = (224, 224)
def ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    ela_image = ImageChops.difference(image, temp_image)
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    return ela_image
def prepare_image(image_path):
    return np.array(ela_image(image_path, 90).resize(image_size)).flatten() / 255.0


def select_image():
    global panelA, panelB, path
    panelA = None
    path = None
    path = filedialog.askopenfilename()
    if len(path) > 0:
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        panelA = Label(image=image)
        panelA.pack_forget()
        panelA.image = image
        panelA.pack(side="top", padx=10, pady=10)
def processing(cad):
    global path,T
    image_path = path 
    image = prepare_image(image_path)
    image = image.reshape(-1, 224, 224, 3)
    y_pred = model.predict(image)
    y_pred_class = np.argmax(y_pred, axis = 1)[0]
    T = Text(root, height = 1, width = 45) 
    Fact =f'Class: {class_names[y_pred_class]} Accuracy in percentage: {np.amax(y_pred) * 100:0.2f}'
    T.place(x=190,y=35)
    T.insert(END, Fact) 

root = Tk()
root.geometry("680x750")
root.title("Fake And Real Image Detection by Using CNN on Relu")
cad = Canvas(root)

cad.create_text(200, 12, text="Orignal Image", font=("Times", 12, "bold"))

cad.pack()
panelA = None
panelB = None
path = None
T = None
def delete_image():
    global panelA,T
    T.destroy()
    panelA.destroy()
    
btn = Button(root, text="Select an image", command=select_image)
btn1 = Button(root, text="Process To find Fake or Real", command=lambda:processing(cad))
btn2 = Button(root, text="Delete Image", command=lambda:delete_image())
btn2.pack(side="bottom", fill="both", expand="no", padx="10", pady="10")
btn1.pack(side="bottom", fill="both", expand="no", padx="10", pady="10")
btn.pack(side="bottom", fill="both", expand="no", padx="10", pady="10")
root.mainloop()


hello


OSError: SavedModel file does not exist at: trained_model_fake_real.h5/{saved_model.pbtxt|saved_model.pb}